<a href="https://colab.research.google.com/github/tripletk/Pepp-Cov19-Net/blob/main/peppcov19net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detecting COVID-19 with Chest X Ray using PyTorch

Importing Libraries

In [6]:
%matplotlib inline

import os
import shutil
import random
import torch
import torchvision
import numpy as np

from PIL import Image
from matplotlib import pyplot as plt

torch.manual_seed(0)

print('Using PyTorch version', torch.__version__)

Using PyTorch version 1.8.1+cu101


Preapring Training and Test Sets

In [5]:
class_names = ['normal', 'viral', 'covid']
root_dir = 'COVID-19 Radiography Database'
source_dirs = ['NORMAL', 'Viral Pneumonia', 'COVID-19']

if os.path.isdir(os.path.join(root_dir, source_dirs[1])):
    os.mkdir(os.path.join(root_dir, 'test'))

    for i, d in enumerate(source_dirs):
        os.rename(os.path.join(root_dir, d), os.path.join(root_dir, class_names[i]))

    for c in class_names:
        os.mkdir(os.path.join(root_dir, 'test', c))

    for c in class_names:
        images = [x for x in os.listdir(os.path.join(root_dir, c)) if x.lower().endswith('png')]
        selected_images = random.sample(images, 30)
        for image in selected_images:
            source_path = os.path.join(root_dir, c, image)
            target_path = os.path.join(root_dir, 'test', c, image)
            shutil.move(source_path, target_path)

Creating Custom Dataset

In [7]:
class ChestXRayDataset(torch.utils.data.Dataset):
  def __init__(self, image_dirs, transform):
    def get_images(class_name):
      images = [x for x in os.listdir(image_dors[class_name]) if x.lower().endswith('png')]
      print(f'Found {len(images)} {class_name} examples')
      return images

    self.images = {}
    self.class_names = ['normal', 'viral', 'covid']

    for c in self.class_names:
      self.images[c] = get_images(c)
    
    self.image_dirs = image_dirs
    self.transform = transform

  def __len__(self):
    return sum([len(self.images[c]) for c in self.class_names])

  def __getitem__(self, index):
    class_name = random.choice(self.class_names)
    index = index % len(self.images[class_name]) # avoid out of bounds index values of different classes
    image_name = self.images[class_name][index]
    image_path = os.path.join(self.image_dirs[class_name], image_name)
    image = Image.open(image_path).convert('RGB') # Using ResNet18 so have to convert image to RGB
    return self.transform(image), self.class_names.index(class_name)
